In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from math import sqrt
import pickle

## Example for training of the model ECIF6::LD

### Data preparation

In [2]:
ecif = pd.read_csv("Descriptors/ECIF_6.0.csv") # Load ECIF (Compressed File)
ligand_descriptors = pd.read_csv("Descriptors/RDKit_Descriptors.csv") # Load ligand descriptors
binding_data = pd.read_csv("Descriptors/BindingData.csv") # Load binding affinity data

# Merge descriptors
ecif = ecif.merge(ligand_descriptors, left_on="PDB", right_on="PDB")
ecif = ecif.merge(binding_data, left_on="PDB", right_on="PDB")
ecif.head()

,PDB,C;4;1;3;0;0-Br;1;1;0;0;0,C;4;1;3;0;0-C;3;3;0;1;1,C;4;1;3;0;0-C;4;1;1;0;0,C;4;1;3;0;0-C;4;1;2;0;0,C;4;1;3;0;0-C;4;1;3;0;0,C;4;1;3;0;0-C;4;2;0;0;0,C;4;1;3;0;0-C;4;2;1;0;0,C;4;1;3;0;0-C;4;2;1;0;1,C;4;1;3;0;0-C;4;2;1;1;1,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_urea,SET,pK
0,10gs,0,0,0,0,0,0,0,0,9,...,0,0,0,0,0,0,0,0,Train,6.40
1,11gs,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,Train,5.82
2,13gs,0,0,0,0,0,0,0,0,17,...,1,0,0,0,0,0,0,0,Train,4.62
3,16pk,0,0,0,0,0,0,0,0,9,...,0,0,0,0,0,0,0,0,Train,5.22
4,184l,0,0,0,0,15,0,0,0,44,...,0,0,0,0,0,0,0,0,Train,4.72


In [3]:
# Split training and test sets
x_train = ecif[ecif["SET"] == "Train"][list(ecif.columns)[1:-2]]
y_train = ecif[ecif["SET"] == "Train"]["pK"]

x_test = ecif[ecif["SET"] == "Test"][list(ecif.columns)[1:-2]]
y_test = ecif[ecif["SET"] == "Test"]["pK"]

print(x_train.shape[0], x_test.shape[0])

9299 285


### Random Forest

In [4]:
RF = RandomForestRegressor(random_state=1206, n_estimators=500, n_jobs=8, oob_score=True, max_features=0.33)
RF.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=0.33, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=8, oob_score=True,
                      random_state=1206, verbose=0, warm_start=False)

In [5]:
y_pred_RF = RF.predict(x_test)
print("Pearson correlation coefficient for RF: ", pearsonr(y_test,y_pred_RF)[0])
print("RMSE for RF:", sqrt(mean_squared_error(y_test,y_pred_RF)))

Pearson correlation coefficient for RF:  0.8281494577006487
RMSE for RF: 1.3478785313031998


### Gradient Boosting Trees

In [6]:
GBT = GradientBoostingRegressor(random_state=1206, n_estimators=20000, max_features="sqrt", max_depth=8, min_samples_split=3, learning_rate=0.005, loss="ls", subsample=0.7)
GBT.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.005, loss='ls',
                          max_depth=8, max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=3,
                          min_weight_fraction_leaf=0.0, n_estimators=20000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=1206, subsample=0.7, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [7]:
y_pred_GBT = GBT.predict(x_test)
print("Pearson correlation coefficient for GBT: ", pearsonr(y_test,y_pred_GBT)[0])
print("RMSE for GBT:", sqrt(mean_squared_error(y_test,y_pred_GBT)))

Pearson correlation coefficient for GBT:  0.8659222014574945
RMSE for GBT: 1.1689978852645881


### Saving the model


In [8]:
pickle.dump(GBT, open("ECIF6_LD_GBT.pkl", 'wb'))